# Create the Channel Means Feature set

In [1]:
# import toolboxes
import pandas as pd 
import numpy as np
import os
from collections import Counter

In [3]:
directory = 'source_data/'
filename = 'EEG_data.csv'

df = pd.read_csv(directory+filename)

In [4]:
df.head()

,version https://git-lfs.github.com/spec/v1
0,oid sha256:7e5a954e0babc650235d488e0d9ae79c4b3...
1,size 4808193


In [5]:
df.shape

(2, 1)

In [6]:
print('Below are subject IDs and their counts')
Counter(df['SubjectID'])

Below are subject IDs and their counts


KeyError: 'SubjectID'

In [ ]:
print('Below are subject IDs and their counts')
Counter(df['VideoID'])

Below are subject IDs and their counts


Counter({0.0: 1412,
         1.0: 1414,
         2.0: 1274,
         3.0: 1206,
         4.0: 1356,
         5.0: 1230,
         6.0: 1181,
         7.0: 1177,
         8.0: 1280,
         9.0: 1281})

In [7]:
# sample rate of 2 Hz (1 sample every 0.5 seconds)
# so it's basically time series data 
# data's kaggle site says theta seems to be where confusion is held 
# this is so stupid they just used the means of each channel as their features 
# need to seperate the combos of videos and participants 

In [8]:
# create an empty numpy array 
eeg_arr = np.zeros((100,14))
eeg_arr.shape

(100, 14)

In [9]:
eeg_arr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
counter = 0
for i in range(10):
    for j in range(10):
        curr = df.loc[(df['user-definedlabel'] == i) & (df['VideoID'] == j)].copy() #SubjectID, VideoID 
        curr_attention = np.mean([curr['Attention']])
        curr_mediation = np.mean([curr['Mediation']])
        curr_raw = np.mean([curr['Raw']])
        curr_delta = np.mean([curr['Delta']])
        curr_theta = np.mean([curr['Theta']])
        curr_alpha1 = np.mean([curr['Alpha1']])
        curr_alpha2 = np.mean([curr['Alpha2']])
        curr_beta1 = np.mean([curr['Beta1']])
        curr_beta2 = np.mean([curr['Beta2']])
        curr_gamma1 = np.mean([curr['Gamma1']])
        curr_gamma2 = np.mean([curr['Gamma2']])
        label = np.mean(curr['user-definedlabel'])
        
        
        eeg_arr[counter,:] = [i,j,label,curr_attention,curr_mediation,curr_raw,curr_delta, curr_theta,
                              curr_alpha1,curr_alpha2, curr_beta1, curr_beta2, curr_gamma1, curr_gamma2]
        # Attention Mediation Raw Delta Theta Alpha1 Alpha2 Beta1 Beta2 Gamma1 Gamma2
        counter +=1 

/home/codespace/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/codespace/.local/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [19]:
eeg_arr

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1., nan, ..., nan, nan, nan],
       [ 0.,  2., nan, ..., nan, nan, nan],
       ...,
       [ 9.,  7., nan, ..., nan, nan, nan],
       [ 9.,  8., nan, ..., nan, nan, nan],
       [ 9.,  9., nan, ..., nan, nan, nan]])

In [20]:
df = pd.DataFrame(eeg_arr, columns = ['SubjectID', 'VideoID', 'user-definedlabel', 'Attention', 
                                      'Mediation', 'Raw', 'Delta', 'Theta', 'Alpha1', 'Alpha2', 
                                      'Beta1', 'Beta2', 'Gamma1', 'Gamma2'])

In [21]:
df.head()

,SubjectID,VideoID,user-definedlabel,Attention,Mediation,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
#save to a CSV so you don't have to keep running this each time 
df.to_csv('cleaned_data/meanfeatures.csv')